<h1><center>DS Capstone Project - Barcelona vs Paris - Let the battle begin.</center></h1><h3><center>Barcelona vs Paris - How (dis)similar are they?</center></h3>

<h4>1. Problem and discussion of the background.</h4>

Barcelona and Paris are two of my most favourite cities in Europe. But why is it that I like these cities the most? Are they similar or very dissimilar. Both cities have different backgrounds, but what makes them so popular.Let's find out if we can use data to solve my problem and to find out which city should be chosen for the next visit.

<h4>2. How will data be used to solve the problem. </h4>

Now we have chosen to use data to find out the (dis)similarities between the two cities, we need to decide which/what data to use. Let's use data from Wikipedia and FourSquare and find out if they can help us to find the answer!
To specify our quest a little bit more we will look for venues and more specifically the number of Vietnamese, Chinese, and Japanese restaurants in both cities as well as list down the 10 most common venues in both cities Neighbourhood wise. I have chosen these types of cuisines as I have an South-East Asian background. Foursquare is a perfect provider to supply us this data.
Wikipedia will be used for getting a start on geospatial information on the districts of Barcelona and arrondissements of Paris.
The outcome of the study will be used to determine to which city the next visit will be organized.

<h4>3. Let the battle begin. </h4>

In [1]:
! pip install geopy

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [3]:
## Foursquare ID
CLIENT_ID = 'NDGX4RBYCYKMWHJNX2MALLNW4ETTWAXUP3D54BN3N5MEGBVJ' # your Foursquare ID
CLIENT_SECRET = 'LUXBWPXTLFIEYSOGH4JI5TM4MBWS5010VBHZ2J243HQJMC10' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

! pip install geopy<h4>3.1 Show Barcelona and the boroughs on the map. </h4>

In [4]:
!wget -q -O 'barcelona_districts_geodata.csv'  "https://raw.githubusercontent.com/AQuynhVo/Quynh_Repo/master/DS_Capstone_Project/barcelona_districts_geodata.csv"
print('Data downloaded!')

Data downloaded!


In [5]:
bcn_data = pd.read_csv('barcelona_districts_geodata.csv', sep=';')

bcn_data.head()

,District,Neighbourhood,Latitude,Longitude
0,Ciutat Vella,La Barceloneta,41.379889,2.189361
1,Ciutat Vella,El Gòtic,41.382778,2.176944
2,Ciutat Vella,El Raval,41.379722,2.168056
3,Ciutat Vella,"Sant Pere, Santa Caterina i la Ribera",41.384608,2.182717
4,Eixample,L'Antiga Esquerra de l'Eixample,41.390000,2.155000


In [6]:
bcn_address = 'Barcelona, ES'

geolocator = Nominatim(user_agent="bcn_explorer")
bcn_location = geolocator.geocode(bcn_address)
bcn_latitude = bcn_location.latitude
bcn_longitude = bcn_location.longitude
print('The geograpical coordinate of Barcelona (ES) are {}, {}.'.format(bcn_latitude, bcn_longitude))

The geograpical coordinate of Barcelona (ES) are 41.3828939, 2.1774322.


In [7]:
# create map of Barcelona using latitude and longitude values
map_barcelona = folium.Map(location=[bcn_latitude, bcn_longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(bcn_data['Latitude'], bcn_data['Longitude'], bcn_data['District'], bcn_data['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_barcelona)  
    
    
# display map
map_barcelona

<h4>3.2 Show Paris and the boroughs on the map. </h4>

Paris has in total 20 boroughs (called arrondissements in French) and 80 neighborhoods. To explore, analyze and segment neighborhoods, longitude and latitude of each neighbor and borough will be added.

This dataset exists for free on the web.

Links to the dataset are:

- https://www.data.gouv.fr/fr/datasets/r/e88c6fda-1d09-42a0-a069-606d3259114e
- https://opendata.paris.fr/explore/dataset/quartier_paris/download/?format=json&timezone=Europe/Berlin

In [8]:
!wget -q -O 'paris_data.json'  "https://opendata.paris.fr/explore/dataset/quartier_paris/download/?format=json&timezone=Europe/Berlin"
print('Data downloaded!')

Data downloaded!


In [9]:
with open('paris_data.json') as paris_json_data:
    paris_data = json.load(paris_json_data)

##paris_data

#### Tranform the data into a *pandas* dataframe

The next task is essentially transforming this data of nested Python dictionaries into a *pandas* dataframe. So let's start by creating an empty dataframe.

In [10]:
#paris_data[1]

# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
paris_neighborhoods = pd.DataFrame(columns=column_names)

for data in paris_data:
    borough = neighborhood_name = data['fields']['c_ar'] 
    neighborhood_name = data['fields']['l_qu']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    paris_neighborhoods = paris_neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
    
paris_neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,4,Saint-Gervais,48.855719,2.358162
1,7,Saint-Thomas-d'Aquin,48.855263,2.325588
2,10,Porte-Saint-Denis,48.873618,2.352283
3,1,Saint-Germain-l'Auxerrois,48.860650,2.334910
4,19,Villette,48.887661,2.374468


In [11]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(paris_neighborhoods['Borough'].unique()),
        paris_neighborhoods.shape[0]
    )
)

The dataframe has 20 boroughs and 80 neighborhoods.


In [12]:
par_address = 'Paris, FR'

par_geolocator = Nominatim(user_agent="par__explorer")
par_location = par_geolocator.geocode(par_address)
par_latitude = par_location.latitude
par_longitude = par_location.longitude
print('The geograpical coordinate of Paris (FR) are {}, {}.'.format(par_latitude, par_longitude))

The geograpical coordinate of Paris (FR) are 48.8566969, 2.3514616.


In [13]:
# create map of Toronto using latitude and longitude values
map_paris = folium.Map(location=[par_latitude, par_longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(paris_neighborhoods['Latitude'], paris_neighborhoods['Longitude'], paris_neighborhoods['Borough'], paris_neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_paris)  
    
map_paris